# Construirea agenților AI cu memorie persistentă folosind Cognee

Acest notebook demonstrează cum să construiești agenți AI inteligenți cu capacități sofisticate de memorie utilizând [**cognee**](https://www.cognee.ai/) - o memorie AI open source care combină grafuri de cunoștințe, căutare semantică și gestionarea sesiunilor pentru a crea sisteme AI conștiente de context.

## 🎯 Obiective de învățare

La finalul acestui tutorial, vei înțelege cum să:
- **Construiești grafuri de cunoștințe bazate pe embeddings**: Transformă textul nestructurat în cunoștințe structurate și interogabile
- **Implementezi memoria sesiunii**: Creează conversații multi-turn cu retenție automată a contextului
- **Păstrezi conversațiile**: Stochează opțional interacțiunile importante în memoria pe termen lung pentru referințe viitoare
- **Interoghezi folosind limbaj natural**: Accesează și utilizează contextul istoric în conversații noi
- **Vizualizezi memoria**: Explorează relațiile din graful de cunoștințe al agentului tău


## 🏗️ Ce vei construi

În acest tutorial, vom crea un **Asistent de Programare** cu memorie persistentă care:

### 1. **Construirea bazei de cunoștințe**
   - Preia informații despre profilul și expertiza dezvoltatorului
   - Procesează principii și bune practici de programare în Python
   - Stochează conversații istorice între dezvoltatori și asistenți AI

### 2. **Conversații conștiente de sesiune**
   - Menține contextul pe parcursul mai multor întrebări din aceeași sesiune
   - Cachează automat fiecare pereche întrebare/răspuns pentru o recuperare eficientă
   - Oferă răspunsuri coerente și contextuale bazate pe istoricul conversației

### 3. **Memorie pe termen lung**
   - Păstrează conversațiile importante într-o memorie pe termen lung
   - Recuperează amintiri relevante din baza de cunoștințe și sesiunile anterioare pentru a informa noile interacțiuni
   - Construiește o bază de cunoștințe în continuă creștere care se îmbunătățește în timp

### 4. **Recuperare inteligentă a memoriei**
   - Utilizează căutarea semantică bazată pe grafuri pentru a găsi informații relevante în toate cunoștințele stocate
   - Filtrează căutările pe subgrupuri de date (informații despre dezvoltatori vs. principii)
   - Combină mai multe surse de date pentru a oferi răspunsuri cuprinzătoare


## 📋 Cerințe preliminare și configurare

### Cerințe de sistem

Înainte de a începe, asigură-te că ai:

1. **Mediu Python**
   - Python 3.9 sau o versiune mai recentă
   - Mediu virtual (recomandat)
   
2. **Redis Cache** (Necesar pentru gestionarea sesiunilor)
   - Redis local: `docker run -d -p 6379:6379 redis`
   - Sau folosește un serviciu Redis gestionat
   
3. **Acces API LLM**
   - Cheie API OpenAI sau alți furnizori (vezi [documentația](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Configurare bază de date**
   - Nicio configurare necesară în mod implicit. Cognee folosește baze de date bazate pe fișiere (LanceDB și Kuzu)
   - Opțional, poți configura Azure AI Search ca un magazin vectorial (vezi [documentația](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Configurare mediu

Creează un fișier `.env` în directorul proiectului tău cu următoarele variabile:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Înțelegerea Arhitecturii Memoriei Cognee

### Cum funcționează Cognee

Cognee oferă un sistem de memorie sofisticat care depășește stocarea simplă de tip cheie-valoare:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Componente cheie:

1. **Grafic de cunoștințe**: Stochează entități, relații și conexiuni semantice
2. **Vectori de încorporare**: Permite căutarea semantică în toate informațiile stocate
3. **Cache de sesiune**: Menține contextul conversației în cadrul și între sesiuni
4. **NodeSets**: Organizează datele în categorii logice pentru recuperare direcționată

### Tipuri de memorie în acest tutorial:

- **Memorie persistentă**: Stocare pe termen lung în graficul de cunoștințe
- **Memorie de sesiune**: Context temporar al conversației în cache-ul Redis
- **Memorie semantică**: Căutare bazată pe similaritate vectorială în toate datele


## 📦 Instalare Pachete Necesare

Instalează Cognee cu suport Redis pentru gestionarea sesiunilor:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inițializați Mediul și Încărcați Bibliotecile

Asigurați-vă că:
1. Redis este pornit (de exemplu, prin Docker: `docker run -d -p 6379:6379 redis`)
2. Variabilele de mediu sunt setate înainte de a importa modulele de cache
3. Dacă este necesar, reporniți kernelul și rulați celulele în ordine

Celula următoare va:
1. Încărca variabilele de mediu din `.env`
2. Configura Cognee cu setările dvs. LLM
3. Activa cache-ul pentru gestionarea sesiunilor
4. Valida că toate componentele sunt conectate corect


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Configurarea Directoarelor de Stocare

Cognee utilizează două directoare separate pentru operațiunile sale:
- **Rădăcina Datelor**: Stochează documentele preluate și datele procesate
- **Rădăcina Sistemului**: Conține baza de date a grafului de cunoștințe și metadatele sistemului

Vom crea directoare izolate pentru acest tutorial, după cum urmează:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Resetarea stării memoriei

Înainte de a începe să construim sistemul de memorie, să ne asigurăm că pornim de la zero.

> 💡 **Sfat**: Puteți sări peste acest pas dacă doriți să păstrați amintirile existente din rulările anterioare când utilizați acest notebook mai târziu.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Partea 1: Construirea bazei de cunoștințe

### Surse de date pentru asistentul nostru pentru dezvoltatori

Vom utiliza trei tipuri de date pentru a crea o bază de cunoștințe cuprinzătoare:

1. **Profilul dezvoltatorului**: Expertiza personală și experiența tehnică
2. **Cele mai bune practici în Python**: Zen-ul Python cu linii directoare practice
3. **Conversații istorice**: Sesiuni anterioare de întrebări și răspunsuri între dezvoltatori și asistenți AI

Aceste date diverse permit agentului nostru să:
- Înțeleagă contextul tehnic al utilizatorului
- Aplice cele mai bune practici în recomandări
- Învățe din interacțiunile de succes anterioare


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Procesează Datele într-un Grafic de Cunoștințe

Acum vom transforma textul brut într-o memorie structurată. Acest proces:

1. **Adaugă date în NodeSets**: Organizează informațiile în categorii logice
   - `developer_data`: Profilul și conversațiile dezvoltatorului
   - `principles_data`: Cele mai bune practici și ghiduri Python

2. **Rulează Cognify Pipeline**: Extrage entități, relații și creează embeddings
   - Identifică conceptele cheie
   - Creează conexiuni semantice între informațiile corelate
   - Generează vector embeddings

Acest proces poate dura câteva momente, pe măsură ce LLM procesează textul și construiește structura graficului:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Vizualizați Graficul de Cunoștințe

Haideți să explorăm structura graficului nostru de cunoștințe. Vizualizarea arată:
- **Noduri**: Entități extrase din text (concepte, tehnologii, persoane)
- **Muchii**: Relații și conexiuni între entități
- **Clustere**: Concepte înrudite grupate după similaritate semantică

Deschideți fișierul HTML generat în browserul dvs. pentru a explora graficul interactiv:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Îmbogățește Memoria cu Memify

Funcția `memify()` analizează graful de cunoștințe și generează reguli inteligente despre date. Acest proces:
- Identifică tipare și bune practici
- Creează linii directoare aplicabile bazate pe conținut
- Stabilește relații între diferite domenii de cunoștințe

Aceste reguli ajută agentul să ia decizii mai informate atunci când răspunde la întrebări. Capturarea unei a doua vizualizări te ajută să compari cum se densifică graful odată ce este îmbogățit.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Partea 2: Recuperarea Inteligentă a Memoriei

### Demonstrația 1: Integrarea Cunoștințelor din Documente Multiple

Acum că graful nostru de cunoștințe este construit, să testăm cum Cognee combină informații din surse multiple pentru a răspunde la întrebări complexe.

Prima interogare demonstrează:
- **Înțelegere semantică**: Găsirea conceptelor relevante chiar și atunci când nu sunt menționate explicit
- **Referințe încrucișate**: Combinarea profilului dezvoltatorului cu principiile Python
- **Raționament contextual**: Aplicarea celor mai bune practici la implementări specifice

### Demonstrația 2: Căutare Filtrată cu NodeSets

A doua interogare arată cum să vizezi subseturi specifice ale grafului de cunoștințe:
- Folosește parametrul `node_name` pentru a căuta doar în `principles_data`
- Oferă răspunsuri concentrate dintr-un domeniu specific de cunoștințe
- Util pentru momentele în care ai nevoie de informații specifice unui domeniu


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Partea 3: Configurarea gestionării sesiunii

### Activarea memoriei conversației

Gestionarea sesiunii este esențială pentru menținerea contextului pe parcursul mai multor interacțiuni. Aici vom:

1. **Inițializa contextul utilizatorului**: Creați sau recuperați un profil de utilizator pentru urmărirea sesiunii
2. **Configurați motorul de cache**: Conectați-vă la Redis pentru stocarea istoricului conversațiilor
3. **Activați variabilele de sesiune**: Configurați variabile de context care persistă între interogări

> ⚠️ **Important**: Acest lucru necesită ca Redis să fie activ și `CACHING=true` în mediul dumneavoastră


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Funcție Utilitară: Vizualizare Istoric Sesiune

Această funcție utilitară ne permite să inspectăm istoricul conversațiilor stocat în Redis. Este utilă pentru:
- Depanarea gestionării sesiunilor
- Verificarea faptului că conversațiile sunt memorate în cache
- Înțelegerea contextului disponibil pentru agent


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sesiunea 1: Laborator de Suport Async — Prima Întrebare

Începe sesiunea `async-support-lab` întrebând despre modele asyncio prietenoase cu telemetria pentru un scraper web masiv. Graful deja cunoaște despre asyncio, aiohttp și practicile de monitorizare, așa că răspunsul ar trebui să reflecte conversațiile anterioare, adaptându-se în același timp la noua întrebare.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Inspectați Memoria Sesiunii 1 După Primul Schimb

Executarea comenzii `show_history(session_1)` imediat după întrebarea inițială confirmă că Cognee a scris atât promptul, cât și completarea în Redis. Ar trebui să vedeți o singură intrare cu ghidajul de concurență.


In [ ]:
await show_history(session_1)

## Sesiunea 1: Urmărire a modelelor de date

Următorul pas este să întrebăm: „Când ar trebui să aleg dataclasses în loc de Pydantic?” folosind același ID de sesiune. Cognee ar trebui să combine principiile Python cu conversațiile anterioare despre FastAPI pentru a oferi sfaturi detaliate—demonstrând că contextul se păstrează într-o sesiune denumită.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Confirmă că Istoricul Sesiunii 1 Conține Ambele Rânduri

O altă apelare `show_history(session_1)` ar trebui să dezvăluie două intrări de tip Întrebare&Răspuns. Acest lucru corespunde pasului de "reluare a memoriei" din laboratorul Mem0 și dovedește că rândurile suplimentare extind același transcript.


In [ ]:
await show_history(session_1)

## Sesiunea 2: Fir de Revizuire a Designului — Sesiune Nouă

Pentru a demonstra izolarea între fire, inițiem `design-review-session` și solicităm îndrumări pentru jurnalizarea revizuirilor incidentelor. Deși baza de cunoștințe de bază este aceeași, noul id de sesiune păstrează transcrierile separate.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Revizuirea Sesiunii 2 Istorie

`show_history(session_2)` ar trebui să listeze doar perechea de solicitare/răspuns din revizuirea designului. Compară aceasta cu Sesiunea 1 pentru a evidenția modul în care Cognee păstrează transcrieri independente în timp ce reutilizează graful de cunoștințe partajat.


In [ ]:
await show_history(session_2)

## Rezumat

Felicitări! Tocmai i-ai oferit asistentului tău de codare un strat de memorie pe termen lung, alimentat de Cognee.

În acest tutorial, ai luat conținut brut de dezvoltator (cod, documentație, conversații) și l-ai transformat într-un grafic + memorie vectorială pe care agentul tău o poate căuta, analiza și îmbunătăți continuu.

Ce ai învățat

1. **De la text brut la memorie AI**: Cum Cognee preia date nestructurate și le transformă într-o memorie inteligentă, ușor de căutat, folosind o arhitectură combinată de vectori + grafic de cunoștințe.

2. **Îmbogățirea graficului cu memify**: Cum să depășești crearea de grafice de bază și să folosești memify pentru a adăuga fapte derivate și relații mai bogate peste graficul existent.

3. **Strategii multiple de căutare**: Cum să interoghezi memoria folosind diferite tipuri de căutare (Q&A bazat pe grafic, completare în stil RAG, perspective, fragmente brute, căutare de cod etc.), în funcție de nevoile agentului tău.

4. **Explorare vizuală**: Cum să inspectezi și să depanezi ceea ce a construit Cognee folosind vizualizări de grafice și interfața Cognee, astfel încât să poți vedea efectiv cum este structurată cunoașterea.

5. **Memorie sensibilă la sesiune**: Cum să combini contextul fiecărei sesiuni cu memoria semantică persistentă, astfel încât agenții să poată reține informații între sesiuni fără a scăpa date între utilizatori.


## Idei principale
1. Memorie ca un Grafic de Cunoștințe susținut de Embeddings

    - **Înțelegere structurată**: Cognee combină un depozit vectorial și un depozit grafic, astfel încât datele tale să fie atât căutabile după semnificație, cât și conectate prin relații. Cognee folosește baze de date bazate pe fișiere în mod implicit (LanceDB pentru vectori, Kuzu pentru baza de date grafică).

    - **Recuperare conștientă de relații**: Răspunsurile pot fi fundamentate nu doar pe „text similar,” ci și pe modul în care entitățile se relaționează.

    - **Memorie vie**: Stratul de memorie evoluează, crește și rămâne interogabil ca un grafic conectat.

2. Moduri de Căutare și Raționament
    - **Recuperare hibridă**: căutarea combină similaritatea vectorială, structura graficului și raționamentul LLM, de la căutarea brută a fragmentelor până la răspunsuri la întrebări conștiente de grafic.

    - **Adaptează modul la sarcină**: Folosește moduri de tip completare când dorești răspunsuri în limbaj natural și moduri de fragment/sumar/grafic când agentul tău are nevoie de context brut sau să-și conducă propriul raționament.

3. Agenți Personalizați, Conștienți de Sesiune
    - **Context de sesiune + memorie pe termen lung**: Cognee păstrează contextul pe termen scurt al „firului” separat de memoria pe termen lung, la nivel de utilizator sau organizație.

## Aplicații în Lumea Reală

1. **Agenți AI Verticali**

    Folosește modelul din acest notebook pentru a alimenta copiloți inteligenți pe domenii care se bazează pe Cognee ca nucleu pentru recuperare și raționament:

- **Copiloți pentru dezvoltatori**: Revizuirea codului, analiza incidentelor și asistenți pentru arhitectură care traversează cod, API-uri, documente de design și tichete ca un singur grafic de memorie.

- **Copiloți orientați către clienți**: Agenți de suport sau succes care extrag informații din documentația produsului, FAQ-uri, notițe CRM și tichete anterioare cu recuperare conștientă de grafic și răspunsuri citate.

- **Copiloți experți interni**: Asistenți pentru politici, juridic sau securitate care raționează pe baza regulilor, ghidurilor și deciziilor istorice interconectate, în loc de PDF-uri izolate.

    Cognee este poziționat explicit ca o memorie persistentă și precisă pentru agenții AI, oferind un grafic de cunoștințe viu care se integrează în spatele agentului tău și înlocuiește combinațiile ad-hoc de depozite vectoriale și coduri grafice personalizate.

2. **Unificarea Silozurilor de Date într-o Singură Memorie**

    Aceeași abordare te ajută să construiești un strat de memorie unificat din surse dispersate:

- **De la silozuri la un singur grafic**: Încarcă date structurate (de ex., baze de date) și nestructurate (de ex., documente, chat-uri) într-un singur grafic susținut de embeddings, în loc de indici separați pentru fiecare sistem.

- **Raționament între surse cu citări**: Rulează raționamente în mai mulți pași peste tot—„alătură” jurnale, metrici și documente prin grafic—și oferă în continuare răspunsuri fundamentate cu proveniență.

- **Hub-uri de cunoștințe**: Pentru domenii precum banking sau educație, Cognee este deja folosit pentru a unifica PDF-uri, sisteme interne și date din aplicații într-un singur grafic de cunoștințe cu vectori, astfel încât agenții să poată răspunde la întrebări cu context precis și citat.

## Pași Următori

Ai implementat bucla de memorie de bază. Iată extensii naturale pe care le poți încerca pe cont propriu (vezi [documentația Cognee](https://docs.cognee.ai/) pentru detalii):

1. **Experimentează cu conștientizarea temporală**: Activează funcția de cognificare temporală pentru a extrage evenimente și marcaje temporale din text.

2. **Introdu raționamentul bazat pe ontologie**: Definește o ontologie OWL pentru domeniul tău. Folosește suportul pentru ontologie al Cognee astfel încât entitățile și relațiile extrase să fie fundamentate în acea schemă, îmbunătățind calitatea graficului și răspunsurile specifice domeniului.

3. **Adaugă un circuit de feedback**: Permite Cognee să ajusteze greutățile muchiilor graficului pe baza feedback-ului real al utilizatorilor, astfel încât recuperarea să se îmbunătățească în timp în loc să rămână statică.

4. **Optimizează pentru personalizare și comportamentul sesiunii**: Folosește ID-uri de utilizator, chiriași și seturi de date pentru a oferi fiecărei persoane sau echipe propria perspectivă asupra motorului de memorie partajat.

5. **Extinde-te la agenți mai complecși**: Conectează Cognee la cadre de agenți pentru a construi sisteme multi-agent care împărtășesc același strat de memorie. *Microsoft Agent Framework x Cognee plugin va fi disponibil în curând.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa maternă ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de oameni. Nu ne asumăm responsabilitatea pentru neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
